In [1]:
import numpy as np
from numba import jit
import time
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import math
from scipy.stats import norm

## Part II: Estimation of Sensitivities in MC

In [2]:
def Delta_Analytical_Call(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1)

def Delta_Analytical_Put(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1) - math.e**-r

In [14]:
@jit(nopython=True, fastmath=True, parallel=False)
def European_Call(T, K, r, S, sigma, trials, z):
    '''
    This function calculates the value of an European Call option
    Arguments: maturity, strike price, interest rate, stock price, volaility, number of trials
    Returns: Array of size trial with values of european puts
    '''
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)

    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*z[i])
        
        if S_cur-K > 0:
            payoff_array[i] = (S_cur-K)*np.exp(-r*T)
        else:
            payoff_array[i] = 0

    return payoff_array


@jit(nopython=True, fastmath=True, parallel=True)
def Digital_Call(T, K, r, S, sigma, trials):
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)
    
    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        
        if S_cur > K:
            payoff_array[i] = 1 * math.e**(-r*T)
        else:
            payoff_array[i] = 0

    return payoff_array

In [15]:
def get_delta_call(kwargs, S, e, seed=None):
    z=np.zeros(kwargs['trials'])
    for i in range(kwargs['trials']):
        z[i] = np.random.normal()
        
    kwargs['z'] = z   
    kwargs['S'] = S
    
    V = European_Call(**kwargs)
    
    if not seed:
        z[i] = np.random.normal()
    kwargs['S'] = S + e
    kwargs['z'] = z   
    
    Ve = European_Call(**kwargs)
    
    return (Ve-V)/ e


def get_delta_digital(kwargs, S, e, seed=None):
    if seed:
        #print("im here")
        np.random.seed(seed)
        
    kwargs['S'] = S
    V = Digital_Call(**kwargs)
    
    
    kwargs['S'] = S + e
    if seed:
        np.random.seed(seed)
    Ve = Digital_Call(**kwargs)
    
    return (Ve-V)/ e

In [16]:
analytical = Delta_Analytical_Call(100, 99, 0.06, 0.2, 1)
analytical

0.6737355117348961

In [17]:
kwargs = {}
kwargs['T'] = 1
kwargs['K'] = 99
kwargs['r'] = 0.06
kwargs['sigma'] = 0.2
kwargs['trials'] = 10**4

In [18]:
S = 100
e = .02

delta_matrix_noseeds = [get_delta_call(kwargs, S, e, None) for x in range(100)]
delta_matrix_seeds = [get_delta_call(kwargs, S, e, 100) for x in range(100)]

In [19]:
abs(analytical - np.mean(delta_matrix_seeds)) / analytical * 100

0.03204893459916989

In [20]:
abs(analytical - np.mean(delta_matrix_noseeds)) / analytical * 100

0.48376400388814944

In [21]:
eps = [0.01,0.02,0.5]
size = [4,5,6,7]

In [22]:
seeded = np.matrix(np.zeros(12))
seeded.shape = (4,3)

unseeded = np.matrix(np.zeros(12))
unseeded.shape = (4,3)

In [23]:
for row in range(len(size)):
    for column in range(len(eps)):
        kwargs['trials'] = 10**size[row]
        e = eps[column]
        seeded[row,column] = abs(analytical - np.mean(get_delta_call(kwargs, S, e, 100))) / analytical * 100
        unseeded[row,column] = abs(analytical - np.mean(get_delta_call(kwargs, S, e,None))) / analytical * 100

In [24]:
unseeded

matrix([[7.23387932e+01, 9.32967178e+00, 9.79485580e-01],
        [2.03024829e-01, 1.20534633e-01, 7.13947266e-01],
        [5.72848366e-01, 3.33950577e-03, 5.88117508e-01],
        [4.49916661e-02, 3.76390118e-02, 6.71144189e-01]])

In [25]:
seeded

matrix([[0.10387494, 0.87137467, 1.23741896],
        [0.04198931, 0.40469068, 0.81516082],
        [0.05252718, 0.07014692, 0.7303867 ],
        [0.03842315, 0.04123337, 0.68228079]])